In [1]:
pip install split-folders

In [2]:
pip install efficientnet-pytorch

  Preparing metadata (setup.py) ... done
  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.7.1-py3-none-any.whl size=16428 sha256=f8fb0ff752475882ee1e9ff1da3a874ba3987eab09c24f0aa5e42e831b104ddb
  Stored in directory: /root/.cache/pip/wheels/03/3f/e9/911b1bc46869644912bda90a56bcf7b960f20b5187feea3baf
Successfully built efficientnet-pytorch


In [ ]:
!pip install wandb -qU

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 35.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.4/195.4 kB 22.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.5/258.5 kB 23.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.4 MB/s eta 0:00:00


In [ ]:
import wandb
wandb.login()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
# import random

# wandb.init(
#     project="pytorch-plant-disease",
#     config={
#         "epochs": 20,
#         "batch_size": 24,
#         "lr": 1e-3,
#         "dropout_rate": random.uniform(0.01, 0.80),
#     }
# )

# config = wandb.config

In [3]:
import os
from glob import glob
from torchvision import transforms, datasets
from torch.utils.data import DataLoader
import splitfolders

def count_images_per_class(data_path):
    classes = os.listdir(data_path)
    print(classes)

    for i in classes:
        new_loc = os.path.join(data_path, i)
        images_jpg = glob(os.path.join(new_loc, '*.jpg'))
        images_JPG =glob(os.path.join(new_loc, '*.JPG'))

        # images_jpg = glob(new_jpg)
        # images_JPG = glob(new_JPG)

        total_images = len(images_jpg) + len(images_JPG)

        print(f"{i}: {total_images} images")

#         wandb.log({f"Class {i} Count": total_images})

def perform_data_split(data_path):
    splitfolders.ratio(data_path, seed=1337, output='train-test-splitted', ratio=(0.6, 0.2, 0.2))
#     wandb.log({"Data Split": "Completed"})

def create_transforms():
    transform_train = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.RandomHorizontalFlip(),
        transforms.RandomVerticalFlip(),
        transforms.ToTensor(),
        transforms.ColorJitter(brightness=0, contrast=0.2, saturation=0, hue=0)
    ])

    transform_tests = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
    ])

    transform_vals = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
    ])

    return transform_train, transform_tests, transform_vals

def create_data_loaders(train_data_dir, test_data_dir, val_data_dir):
    transform_train, transform_tests, transforms_vals = create_transforms()

    train_data = datasets.ImageFolder(train_data_dir, transform=transform_train)
    test_data = datasets.ImageFolder(test_data_dir, transform=transform_tests)
    val_data = datasets.ImageFolder(val_data_dir, transform=transforms_vals)

#     wandb.log({"Train Dataset Size": len(train_data)})
#     wandb.log({"Validation Dataset Size": len(test_data)})

    train_loader = DataLoader(train_data, batch_size=32, drop_last=True, shuffle=True, num_workers=0)
    validation_loader = DataLoader(test_data, batch_size=32, drop_last=True, shuffle=True, num_workers=0)
    test_loader = DataLoader(val_data, batch_size=32, drop_last=True, shuffle=True, num_workers=0)

    return train_loader, validation_loader, test_loader, train_data, test_data

In [4]:
!pip install git+https://github.com/PyTorchLightning/pytorch-lightning
import pytorch_lightning as pl
print(pl.__version__)

  Cloning https://github.com/PyTorchLightning/pytorch-lightning to /tmp/pip-req-build-xsc4ix7r
  Running command git clone --filter=blob:none --quiet https://github.com/PyTorchLightning/pytorch-lightning /tmp/pip-req-build-xsc4ix7r
  Resolved https://github.com/PyTorchLightning/pytorch-lightning to commit b871f7a826db015b8b909cf86302c6c4cbf18ec1
  Running command git submodule update --init --recursive -q
  Encountered 22 file(s) that should have been pointers, but weren't:
        .notebooks/course_UvA-DL/01-introduction-to-pytorch.ipynb
        .notebooks/course_UvA-DL/02-activation-functions.ipynb
        .notebooks/course_UvA-DL/03-initialization-and-optimization.ipynb
        .notebooks/course_UvA-DL/04-inception-resnet-densenet.ipynb
        .notebooks/course_UvA-DL/05-transformers-and-MH-attention.ipynb
        .notebooks/course_UvA-DL/06-graph-neural-networks.ipynb
        .notebooks/course_UvA-DL/07-deep-energy-based-generative-models.ipynb
        .notebooks/course_UvA-DL/08-

In [5]:
import torch.nn as nn
import torch
import pytorch_lightning as pl
from efficientnet_pytorch import EfficientNet

class ModifiedEfficientNet(nn.Module):
    def __init__(self, num_classes, model_name='efficientnet-b0', dropout_rate=0.2):
        super(ModifiedEfficientNet, self).__init__()

        self.efficientnet = EfficientNet.from_pretrained(model_name)

        self.additional_layers = nn.Sequential(
            nn.Conv2d(1280, 128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.Dropout(p=dropout_rate),
        )

        in_features = self.efficientnet._fc.in_features
        self.additional_fc = nn.Sequential(
            nn.Dropout(p=dropout_rate),
            nn.Linear(128, 64),
            nn.ReLU(inplace=True),
            nn.Dropout(p=dropout_rate),
            nn.Linear(64, num_classes)
        )
        self.efficientnet._fc = nn.Identity()

    def forward(self, x):
        x = self.efficientnet.extract_features(x)

        x = self.additional_layers(x)

        x = x.mean([2, 3])

        x = self.additional_fc(x)

        return x

# def get_model(dropout_rate):
#     num_classes = 8
#     model = ModifiedCNN(num_classes, dropout_rate=dropout_rate)
#     return model

# class PlantDiseaseModel(pl.LightningModule):
#   def __init__(self, config):
#     super(PlantDiseaseModel, self).__init__()
#     self.model = ModifiedCNN(num_classes=8, dropout_rate=config.dropout_rate)
#     self.config = config

#   def forward(self, x):
#     return self.model(x)

#   def training_step(self, batch, batch_idx):
#     inputs, labels = batch
#     outputs = self.model(inputs)
#     loss = F.cross_entropy(outputs, labels)
#     return loss

#   def validation_step(self, batch, batch_idx):
#     inputs, labels = batch
#     outputs = self.model(inputs)
#     loss = F.cross_entropy(outputs, labels)
#     preds = torch.argmax(outputs, dim=1)
#     acc = (preds == labels).float().mean()
#     return {'val loss': loss, 'val_acc': acc}

#   def validation_epoch_end(self, outputs):
#     avg_loss = torch.stack([x['val_loss'] for x in outputs]).mean()
#     avg_acc = torch.stack([x['val_acc'] for x in outputs]).mean()
#     self.log('val_loss', avg_loss)
#     self.log('val_acc', avg_acc)

#   def configure_optimizers(self):
#     return torch.optim.Adam(self.parameters(), lr=self.config.lr)

In [6]:
pip install torchmetrics

In [7]:
import pytorch_lightning as pl
import torch
import torch.nn.functional as F
from typing import Any, Optional, Union
import torchmetrics
from torchmetrics import Accuracy
# from src.model.modelCNN import ModifiedCNN

# STEP_OUTPUT = Union[float, torch.Tensor]

class Classifier(pl.LightningModule):
    def __init__(self, n_classes: int = 8) -> None:
        super().__init__()
        self.model = ModifiedEfficientNet(num_classes=n_classes, dropout_rate=0.2)
        self.accuracy = torchmetrics.classification.Accuracy(task="multiclass", num_classes=n_classes)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return self.model(x)

    def training_step(self, batch, batch_idx):
        inputs, labels = batch
        outputs = self.model(inputs)
        loss = F.cross_entropy(outputs, labels)
        return loss

    def validation_step(self, batch, batch_idx):
        inputs, labels = batch
        outputs = self.model(inputs)
        loss = F.cross_entropy(outputs, labels)
        preds = torch.argmax(outputs, dim=1)
        acc = torch.sum(preds == labels).item() / len(labels)
        self.log('val_loss', loss, on_epoch=True, prog_bar=True)
        self.log('val_acc', acc, on_epoch=True, prog_bar=True)

    def test_step(self, batch, batch_idx):
        inputs, labels = batch
        outputs = self.model(inputs)
        loss = F.cross_entropy(outputs, labels)
        preds = torch.argmax(outputs, dim=1)
        acc = self.accuracy(preds, labels)
        self.log('test_loss', loss, on_step=True, on_epoch=True, prog_bar=True)
        self.log('test_acc', acc, on_step=True, on_epoch=True, prog_bar=True)

    # def on_validation_epoch_end(self) -> None:
    #     avg_loss = torch.stack([x['val_loss'] for x in self.outputs]).mean()
    #     avg_acc = torch.stack([x['val_acc'] for x in self.outputs]).mean()
    #     self.log('val_loss', avg_loss)
    #     self.log('val_acc', avg_acc)

    def configure_optimizers(self) -> Any:
        return torch.optim.Adam(self.parameters(), lr=1e-3)

    # def on_validation_epoch_end(self) -> None:
    #     pass


In [10]:
model = Classifier(n_classes=8)
print(model)

Downloading: "https://github.com/lukemelas/EfficientNet-PyTorch/releases/download/1.0/efficientnet-b0-355c32eb.pth" to /root/.cache/torch/hub/checkpoints/efficientnet-b0-355c32eb.pth
100%|██████████| 20.4M/20.4M [00:00<00:00, 142MB/s]


Loaded pretrained weights for efficientnet-b0
Classifier(
  (model): ModifiedEfficientNet(
    (efficientnet): EfficientNet(
      (_conv_stem): Conv2dStaticSamePadding(
        3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False
        (static_padding): ZeroPad2d((0, 1, 0, 1))
      )
      (_bn0): BatchNorm2d(32, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
      (_blocks): ModuleList(
        (0): MBConvBlock(
          (_depthwise_conv): Conv2dStaticSamePadding(
            32, 32, kernel_size=(3, 3), stride=[1, 1], groups=32, bias=False
            (static_padding): ZeroPad2d((1, 1, 1, 1))
          )
          (_bn1): BatchNorm2d(32, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
          (_se_reduce): Conv2dStaticSamePadding(
            32, 8, kernel_size=(1, 1), stride=(1, 1)
            (static_padding): Identity()
          )
          (_se_expand): Conv2dStaticSamePadding(
            8, 32, kernel_siz

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import torch
import torch.nn as nn
import torch
import pytorch_lightning as pl
from typing import Dict
from pytorch_lightning.callbacks import ModelCheckpoint
import wandb
from pytorch_lightning.loggers import WandbLogger
from pathlib import Path

# from scripts.model import get_model
# from dataset.dataset import create_data_loaders, count_images_per_class, perform_data_split

class TrainConfig:
    def __init__(self, dataset_root: Path, max_epochs: int, batch_size: int, optimizer: Dict):
        self.dataset_root = dataset_root
        self.max_epochs = max_epochs
        self.batch_size = batch_size
        self.optimizer = optimizer

def train(cfg: TrainConfig) -> None:
    wandb.finish()

    wandb.init(
        project="plant-disease-efficient",
        config={
            "batch_size": cfg.batch_size,
            "learning_rate": cfg.optimizer["learning_rate"],
            "epochs": cfg.max_epochs,
            "optimizer": cfg.optimizer["type"]
        },
    )

    logger = WandbLogger(name="Wandb", project="plant-disease-efficient")

    checkpoint_callback = ModelCheckpoint(
        dirpath = '/content/scripts/checkpoints',
        filename = "epoch={epoch}-step={global_step}",
    )


    trainer = pl.Trainer(
        max_epochs = cfg.max_epochs,
        log_every_n_steps = 5,
        callbacks = [checkpoint_callback],
        accelerator = "cuda",
        devices = 1,
        strategy = "auto",
        enable_model_summary = True,
        logger = logger,
    )

    model = Classifier(n_classes=8)
    data_path = Path("/content/drive/MyDrive/Plant_Disease_Dataset")


    perform_data_split(data_path)

    # Define data directories
    train_data_dir = '/content/train-test-splitted/train'
    test_data_dir = '/content/train-test-splitted/test'
    val_data_dir = '/content/train-test-splitted/val'

    # Create datasets and data loaders
    train_loader, validation_loader, test_loader, train_data, test_data = create_data_loaders(train_data_dir, test_data_dir, val_data_dir)

    trainer.fit(model, train_loader, validation_loader)
    torch.save(
        model.state_dict(), "/content/Modified_Efficient.pt"
    )

    trainer.test(model, test_loader)

    wandb.finish()


if __name__ == "__main__":
    config_values = {
        "max_epochs": 20,
        "dataset_root": Path("/content/drive/MyDrive/Plant_Disease_Dataset"),
        "batch_size": 32,
        "optimizer": {
            "type": "Adam",
            "learning_rate": 1e-3
        }
    }

    train_cfg = TrainConfig(**config_values)
    train(train_cfg)


INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


Loaded pretrained weights for efficientnet-b0


Copying files: 4004 files [00:29, 135.23 files/s]
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/loggers/wandb.py:390: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name     | Type                 | Params
--------------------------------------------------
0 | model    | ModifiedEfficientNet | 5.5 M 
1 | accuracy | MulticlassAccuracy   | 0     
--------------------------------------------------
5.5 M     Trainable params
0         Non-trainable params
5.5 M     Total params
21.964    Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/connectors/data_connector.py:492: Your `val_dataloader`'s sampler has shuffling enabled, it is strongly recommended that you turn shuffling off for val/test dataloaders.


...

Validation: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=20` reached.
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/connectors/data_connector.py:492: Your `test_dataloader`'s sampler has shuffling enabled, it is strongly recommended that you turn shuffling off for val/test dataloaders.


Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│      test_acc_epoch       │    0.9750000238418579     │
│      test_loss_epoch      │    0.09167923778295517    │
└───────────────────────────┴───────────────────────────┘

epoch,▁▁▂▂▂▃▃▃▄▄▅▅▅▆▆▆▇▇▇██
test_acc_epoch,▁
test_acc_step,▁███▆▆███▆▆▆▆█▆▃▆▆████▃▃▃
test_loss_epoch,▁
test_loss_step,▃▁▂▁▃▇▁▁▁▃▂▂▂▁▂█▄▄▁▁▁▁█▂▃
trainer/global_step,▁▂▂▂▃▃▃▄▄▅▅▆▆▆▇▇██▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_acc,▂▁▆▆██▇██▇███▇████▆█
val_loss,▆█▂▄▁▁▂▁▁▁▁▁▁▂▁▁▁▁▂▁
epoch,20
test_acc_epoch,0.975
test_acc_step,0.9375


In [ ]:
# """
# Implementation of EfficientNet CNN architecture.
# """

# import torch
# import torch.nn as nn
# from math import ceil

# base_model =[
#     [1, 16, 1, 1, 3],
#     [6, 24, 2, 2, 3],
#     [6, 40, 2, 2, 5],
#     [6, 80, 3, 2, 3],
#     [6, 112, 3, 1, 5],
#     [6, 192, 4, 2, 5],
#     [6, 320, 1, 1, 3],
# ]

# phi_values = [
#     "b0": (0.5, 240, 0.2),
#     "b1": (0.5, 240, 0.2),
#     "b2": (1, 260, 0.3),
#     "b3": (2, 300, 0.3),
#     "b4": (3, 380, 0.4),
#     "b5": (4, 456, 0.4),
#     "b6": (5, 528, 0.5),
#     "b7": (6, 600, 0.5),
# ]

# class CNNBlock(nn.Module):
#   def __init__(
#       self, in_channels, out_channels, kernel_size, stride, padding, groups=1
#   ):
#     super(CNNBlock, self).__init__()
#     self.cnn = nn.Conv2d(
#         in_channels,
#         out_channels,
#         kernel_size,
#         stride,
#         padding,
#         groups = groups,
#         bias = False,
#     )
#     self.bn = nn.BatchNorm2d(out_channels)
#     self.silu =  nn.SiLU()
#   def forward(self, x):
#     return self.silu(self.bn(self.cnn(x)))

# class SqueezeExcitation(nn.Module):
#   def __init__(self, in_channels, reduced_dim):
#     super(SqueezeExcitation, self).__init__()
#     self.se = nn.Sequential(
#         nn.AdapativeAvgPool2d(1),
#         nn.Conv2d(in_channels, reduced_dim, 1),
#         nn.SiLU(),
#         nn.Conv2d(reduced_dim, in_channels, 1),
#         nn.Sigmoid(),
#     )
#   def forward(self, x):
#     return x * self.se(x)

# class InvertedResidualBlock(nn.Module):
#   def __init__(
#       self,
#       in_channels,
#       out_channels,
#       kernel_size,
#       stride,
#       padding,
#       expand_ratio,
#       reduction=4,
#       survival_prob=0.8,
#   ):
#       super(InvertedResidualBlock, self).__init__()
#       self.survival_prob = 0.8
#       self.use_residual = in_channels == out_channels and stride == 1
#       hidden_dim = in_channels * expand_ratio
#       self.expand = in_channels != hidden_dim
#       reduced_dim = int(in_channels / reduction)

#       if self.expand:
#         self.expand_conv = CNNBlock(
#             in_channels,
#             hidden_dim,
#             kernel_size=3,
#             stride=1,
#             padding=1
#         )

#         self.conv == nn.Sequential(
#             CNNBlock(
#                 hidden_dim,
#                 hidden_dim,
#                 kernel_size,
#                 stride,
#                 padding,
#                 groups=hidden_dim,
#             ),
#             SqueezeExcitation(hidden_dim, reduced_dim),
#             nn.Conv2d(hidden_dim, out_channels, 1, bias=False),
#             nn.BatchNorm2d(out_channels),
#         )

#   def stochastic_depth(self, x):
#     if not self.training:
#       return x

#       binary_tensor = (
#           torch.rand(x.shape[0], 1, 1, 1, device=x.device) < self.survival_prob
#       )
#       return torch.div(x, self.survival_prob) * binary_tensor

#   def forward(self, inputs):
#     x = self.expand_conv(inputs) if self.expand else inputs

#     if self.use_residual:
#       return self.stochastic_depth(self.conv(x)) + inputs
#     else:
#       return self.conv(x)

# class EfficientNet(nn.Module):
#   def __init__(self, version, num_classes):
#     super(EfficientNet, self).__init__()
#     width_factor